In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/Shared\ drives/ecl-pedro/video-coding-utils/notebooks/

/content/drive/Shared drives/ecl-pedro/video-coding-utils/notebooks


---

In [4]:
from utils.utils import *
from utils import models

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.image import psnr
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
def PSNR(x,y):
    return psnr(x,y,max_val=1)

In [9]:
ds = tf.data.Dataset.from_tensor_slices(list(range(50000)))

In [10]:
ds = ds.shuffle(50000)

In [11]:
def parse_function(xdir, ydir):
    
    def fun(i):
        
        i = tf.strings.as_string(i)
        fcompressed = tf.io.read_file(xdir + i + '.png')
        compressed = tf.image.decode_png(fcompressed, channels=1)
        compressed = tf.image.convert_image_dtype(compressed, tf.float32)
        compressed = tf.image.resize(compressed, [64, 64])
        
        fgroundtruth = tf.io.read_file(ydir + i + '.png')
        groundtruth = tf.image.decode_png(fgroundtruth, channels=1)
        groundtruth = tf.image.convert_image_dtype(groundtruth, tf.float32)
        groundtruth = tf.image.resize(groundtruth, [64, 64])
        
        return compressed, groundtruth
    
    return fun

In [12]:
ds = ds.map(parse_function('../../data/preprocessed/basketball/compressed/', '../../data/preprocessed/basketball/ground_truth/'))

In [13]:
bds = ds.batch(128)

In [14]:
model = models.QECNN_I()

In [15]:
model.build((None, 64, 64, 1))

In [16]:
model.compile(
    optimizer='SGD',
    loss=MSE,
    metrics=[PSNR]
)

In [17]:
ckpt_callback = ModelCheckpoint(
    filepath='checkpoints/QECNN_I',
    save_weights_only=True,
    verbose=0
)

In [18]:
model.load_weights('checkpoints/QECNN_I')

In [1]:
history = model.fit(
    bds,
    epochs=50,
    callbacks=ckpt_callback
)

NameError: ignored